In [ ]:
import dualcodec
import torchaudio
from IPython.display import Audio

In [ ]:
model_id = "12hz_v1"
dualcodec_model_path = "./output_checkpoints/dualcodec_12hz_16384_4096_8vq_scratch/checkpoint/epoch-0000_step-0013800_loss-89.047997-dualcodec_12hz_16384_4096_8vq_scratch"
dualcodec_model = dualcodec.get_model(model_id, dualcodec_model_path)

In [ ]:
dualcodec_inference = dualcodec.Inference(dualcodec_model=dualcodec_model, device="cpu")

In [ ]:
audio, sr = torchaudio.load("audio_samples/tara.wav")
audio = torchaudio.functional.resample(audio, sr, 24000)
audio = audio.reshape(1,1,-1)
audio = audio.cpu()
# extract codes, for example, using 8 quantizers here:
semantic_codes, acoustic_codes = dualcodec_inference.encode(audio, n_quantizers=8)

In [ ]:
out_audio = dualcodec_inference.decode(semantic_codes, acoustic_codes)
Audio(out_audio.cpu().squeeze(0), rate=24000)